# Step 07: Execute Portfolio Mapping Batch

This notebook executes SQL scripts to create sub-portfolios in Moody's Risk Modeler.

**Tasks:**
- Retrieve Portfolio Mapping batch from Stage 01
- Review batch configuration and job details
- Execute SQL mapping scripts for each portfolio
- Display results and update job status

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch
from helpers.database import execute_query
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_07_Execute_Portfolio_Mapping.ipynb')

# Display context
ux.header("Portfolio Mapping Execution")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Portfolio Mapping Batch

In [ ]:
# Retrieve Portfolio Mapping batch from Stage_01/Step_03
ux.subheader("Retrieve Portfolio Mapping Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.PORTFOLIO_MAPPING not in batches:
    raise ValueError(f"Portfolio Mapping batch not found. Available: {list(batches.keys())}")

mapping_batch_id = int(batches[BatchType.PORTFOLIO_MAPPING])

ux.success(f"✓ Retrieved Portfolio Mapping batch: ID={mapping_batch_id}")
step.log(f"Retrieved Portfolio Mapping batch: ID={mapping_batch_id}")

## 3) Review Batch Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(mapping_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch
jobs = get_batch_jobs(mapping_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Show sample configurations
if job_count > 0:
    ux.info("\nPortfolio Mapping Configurations:")
    rows = []
    for job in jobs:
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        if not config_result.empty:
            config = config_result.iloc[0]['job_configuration_data']
            rows.append([
                job['id'],
                config.get('Portfolio', 'N/A'),
                config.get('Database', 'N/A'),
                config.get('Import File', 'N/A'),
            ])
    ux.table(rows, headers=["Job ID", "Portfolio", "Database", "Import File"])

step.log(f"Verified batch: {job_count} jobs ready for execution")

## 4) Execute Portfolio Mapping

In [ ]:
# Execute portfolio mapping SQL scripts
ux.subheader("Execute Portfolio Mapping")

ux.info("")
ux.info("Execution Process:")
ux.info("  - Each job executes a SQL script to create sub-portfolios")
ux.info("  - SQL scripts are located in workspace/sql/portfolio_mapping/")
ux.info("  - Script naming: 2b_Query_To_Create_Sub_Portfolios_{Import File}_RMS_BackEnd.sql")
ux.info("  - This is a synchronous local operation (not submitted to Moody's)")
ux.info("")

# Execute
ux.info("\nExecuting mapping scripts...")

# Pass step.step_id to associate batch with this step (not the creation step)
result = submit_batch(mapping_batch_id, IRPClient(), step_id=step.step_id)

# Display results
ux.success(f"\n✓ Batch execution completed")
ux.info(f"  Executed: {result['submitted_jobs']} jobs")
ux.info(f"  Status: {result['batch_status']}")

# Check for errors
failed_count = len([j for j in result['jobs'] if 'error' in j])
if failed_count > 0:
    ux.warning(f"\n⚠ {failed_count} job(s) failed to execute")
    for job_result in result['jobs']:
        if 'error' in job_result:
            ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

step.log(f"Batch executed: {result['submitted_jobs']} jobs, {failed_count} failed")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

try:
    # Prepare output data
    output_data = {
        'batch_id': mapping_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': result['batch_status'],
        'submitted_jobs': result['submitted_jobs'],
        'failed_jobs': failed_count
    }
    
    # Complete the step
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("✓ PORTFOLIO MAPPING BATCH EXECUTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nExecuted {result['submitted_jobs']} job(s)")
    if failed_count > 0:
        ux.warning(f"⚠ {failed_count} job(s) failed to execute")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNext: Proceed to other data loading steps or analysis")

except Exception as e:
    ux.error(f"✗ Step completion failed: {str(e)}")
    step.fail(str(e))
    raise